## 신경망 (Neural Networks)

`torch.nn` 패키지를 사용하여 신경망을 생성해보도록 하자. `nn`은 모델을 정의하고 미분하기 위해 `autograd`를 사용한다. `nn.Module`은 layer와 output을 반환하는 forward 메서드를 포함하고 있다.

신경망의 일반적인 학습 과정은 다음과 같다.
- 학습 가능한 매개변수(또는 가중치)를 갖는 신경망을 정의
- 데이터셋 입력을 반복 (epoch)
- 입력을 신경망에서 전파한 후 loss 계산
- 변화도(gradient)를 신경망의 매개변수들에 역으로 전파
- 신경망의 가중치 갱신 (ex. 새로운 가중치 = 기존 가중치 - 학습률 * gradient)

### 신경망 정의하기

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [49]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5) # 입력 이미지 채널 1, 출력 채널 6, 5x5 정사각 컨볼루션 행렬
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16*5*5, 120) # affine 연산 / 이전 layer output * 이미지 차원
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # output 10 채널
        
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # (2,2) 크기 윈도우에 대해 맥스 풀링
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x,1) # 배치 차원을 제외한 모든 차원을 하나로 flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [50]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


`forward` 함수를 정의하면 gradient를 계산하는 `backward` 함수는 `autograd`를 사용하여 자동으로 정의된다.

In [51]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1의 .weight

10
torch.Size([6, 1, 5, 5])


이 신경망은 5x5 커널을 사용하는 convolution layer 두 개를 사용하며 두 번의 max pooling을 거쳐 5x5 feature map 형태가 fc1에 전달되어야 한다.

첫 번째 컨볼루션 레이어는 패딩이 없을 때 4 픽셀을 줄이게 되고 맥스 풀링에 의해 절반으로 줄어들게 된다. 두 번째 컨볼루션 레이어와 맥스 풀링도 유사하게 진행된다.

$$(((X - 4)/2)-4)/2 = 5$$
$$(((5*2)+4)*2)+4 = X = 32$$

위와 같은 계산에 의해, 해당 네트워크는 32x32의 입력 크기를 가진다는 것을 알 수 있다. 이제 이 신경망에 임의의 32x32 입력값을 넣어보도록 하자.

In [52]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1100, -0.0458,  0.0222,  0.0944, -0.0625, -0.0594, -0.0460,  0.0826,
         -0.0519,  0.0117]], grad_fn=<AddmmBackward0>)


모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로 역전파를 수행한다.

In [53]:
net.zero_grad()
out.backward(torch.randn(1,10))

#### Note
`torch.nn`은 하나의 샘플이 아닌, 샘플들의 mini-batch만을 입력으로 받도록 지원한다. 

예를 들어, `nn.Conv2D`는 `nSamples x nChannels x Height x Width`의 4차원 Tensor를 입력으로 받는다. 만약 하나의 샘플만 있다면, `input.unsqueeze(0)`를 사용해서 가상의 차원을 추가해야 한다.

#### 요약:
- torch.Tensor - backward() 같은 autograd 연산을 지원하는 다차원 배열 입니다. 또한 tensor에 대한 변화도를 갖고 있습니다.
- nn.Module - 신경망 모듈. 매개변수를 캡슐화(encapsulation)하는 간편한 방법 으로, GPU로 이동, 내보내기(exporting), 불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공합니다.
- nn.Parameter - Tensor의 한 종류로, Module 에 속성으로 할당될 때 자동으로 매개변수로 등록 됩니다.
- autograd.Function - autograd 연산의 순방향과 역방향 정의 를 구현합니다. 모든 Tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor 를 생성하고 이력(history)을 인코딩 하는 함수들과 연결하고 있습니다.

### 손실 함수 (Loss Function)

손실 함수는 (output, target)을 한 쌍의 입력으로 받아 output과 target 사이 차이를 계산한다.

`torch.nn` 패키지에는 여러 가지 손실 함수들을 제공하고 있으며, 여기서는 출력과 대상과의 평균제곱오차(mean-squared error)를 계산하는 간단한 손실 함수인 `nn.MSEloss`를 이용해보자.

In [54]:
output = net(input)
target = torch.randn(10) # 임의의 정답
target = target.view(1,-1) # output과 같은 shape로 변환
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(2.1147, grad_fn=<MseLossBackward0>)


`.grad_fn` 속성을 이용해 loss를 역방향에서 따라가다 보면 다음과 같은 모습의 연산 그래프를 볼 수 있다.

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

In [55]:
print(loss.grad_fn) # MSE Loss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

### 역전파 (Backprop)

오차(error)를 역전파하기 위해서는 `loss.backward()`만 해주면 된다. 기존에 계산된 gradient의 값을 누적시키고 싶지 않다면 기존에 계산된 gradient를 0으로 만드는 작업이 필요하다.

In [56]:
net.zero_grad()

print(f'conv1.bias.grad before backward: {net.conv1.bias.grad}')

loss.backward()

print(f'conv1.bias.grad after backward: {net.conv1.bias.grad}')

conv1.bias.grad before backward: None
conv1.bias.grad after backward: tensor([-0.0189, -0.0151,  0.0172,  0.0185,  0.0196, -0.0281])


### 가중치 갱신

가장 많이 사용되는 단순한 갱신 규칙은 확률적 경사하강법(Stochastic Gradient Descent; SGD)이다. SGD는 매개변수를 업데이트하기 위해 전체 데이터를 사용하는 경사하강법(Gradient Descent)과 달리 매 반복마다 하나 또는 소량의 데이터 샘플(미니 배치)을 사용해서 변화도를 계산한다.

이 방법은 계산 효율성이 높고 한번에 모든 데이터셋을 사용하지 않아도 되기 때문에 대규모 데이터셋에서도 사용할 수 있다는 장점이 있다. 다만 소수의 샘플을 사용하기 때문에 변동성이 높고 업데이트가 불안정할 수 있으며 최적점에 가까워질수록 수렴 속도가 느려진다는 단점이 있다.

이 방식을 간단한 수식으로 나타내면 다음과 같이 나타낼 수 있다.
$$ New\ Weight = Current\ Weight - Gradient * Learning\ Rate$$

이를 파이썬으로 간단하게 다음과 같이 구현할 수 있다.

In [57]:
learning_rate = 1e-4
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

신경망을 구성할 때 SGD, Nesterov-SGD, Adam, RMSProp 등과 같은 다양한 갱신 규칙을 사용하고 싶을 수 있다. 이를 위해서 `torch.optim`이라는 패키지를 사용할 수 있다.

In [58]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=1e-4)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # 업데이트 진행